## Photon Noise components on qubicsoft

Author: M. M. Gamboa Lerena

This notebook aims to show the new structure of the photon noise's code. Previously the components of the noise were computed in a hard-coded way. This means that the total NEP, composed by many terms detailed below, were directly sum up to the final variable. 

This new code initialize a structure through a class called `Noise()` and initialized it considering the components of the instrument. This initiallization is performed using `load_NEP_parameters(scene)`. The new structure of the code encapsulate the calculation of the total noise equivalent power (NEP) for each component.

#### Components (and python's methods): 
1. Before back to back horns via `NEP_before_horns(self, noise, nu, return_only = False, sampling = None)` method,
2. Horn array via `NEP_horns(self, noise, return_only = False, sampling = None)` method, 
3. Environment NEP via `NEP_environment(self, noise, names, return_only = False, sampling = None)` method, 
4. Optical combiner via `NEP_combiner(self, noise, return_only = False, sampling = None)` method, 
5. Cold stop filter via `NEP_coldstop(self, noise, return_only = False, sampling = None)` method, 
6. Dichroic (if appropiate) via `NEP_dichroic(self, noise, return_only = False, sampling = None)` method,
7. Neutral density filter via `NEP_neutraldensityfilter(self, noise, return_only = False, sampling = None)` method, 
8. Two low pass edge filters each computed via `NEP_lowpassedge(self, noise, i, return_only = False, sampling = None)` method, and a 
9. Last filter computed via `NEP_lastfilter(self, noise, return_only = False, sampling = None)` method. This method is different for the 220GHz channel because of the multimode nature of the band. In such case is used `NEP_lastfilters_220(self, noise, return_only = False, sampling = None)`  

Finally the total NEP is computed as before: $\sqrt{\sum_{i=0}^{Ncomps} {\rm NEP}_i^2 + {\rm NEP_{env}^2}}$.


Parameters: 
* `noise` is an instance of the initialized class called `Noise()`. For example: `noise = qinst[0].load_NEP_parameters(scene)`
* `nu` is the frequency of the `QubicInstrument` object
* Using `return_only = True` you get the noise realization for a particular component in an independent way (you need to provide a `qubic.get_sampling()` instance)

In [ ]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
from scipy.stats import norm

import qubic
from pysimulators import FitsArray
from qubic.calibration import QubicCalibration

In [ ]:
mp.rcParams["font.size"] = 18
mp.rcParams["figure.figsize"] = (14,8)

In [ ]:
# Read dictionary
d = qubic.qubicdict.qubicDict()
d.read_from_file('test_photon_noise.dict')
d["config"] = "FI"
d["npointings"] = 1000
d["random_pointing"] = False
d["repeat_pointing"] = True
d["sweeping_pointing"] = False
#
d["photon_noise"] = True
d["noiseless"] = False
#
d["debug"] = True
d["filter_nu"]= 220e9
d["synthbeam"]='CalQubic_Synthbeam_Analytical_FFF_CC.fits'

In [ ]:
d['optics'] = d['optics'].replace(d['optics'][-10:-8], d['config'])
d['optics'] = d['optics'].replace(d['optics'][-7:-4], str(int(d["filter_nu"] /1e9)))
d['detarray'] = d['detarray'].replace(d['detarray'][-7:-5], d['config'])
d['hornarray'] = d['hornarray'].replace(d['hornarray'][-7:-5], d['config'])
if d['beam_shape'] == 'gaussian':
    d['primbeam'] = d['primbeam'].replace(d['primbeam'][-6], '2')
    primary_shape = 'gaussian'
    secondary_shape = 'gaussian'
elif d['beam_shape'] == 'fitted_beam':
    d['primbeam'] = d['primbeam'].replace(d['primbeam'][-6], '3')
    primary_shape = 'fitted_beam'
    secondary_shape = 'fitted_beam'
else:
    d['primbeam'] = d['primbeam'].replace(d['primbeam'][-6], '4')
    primary_shape = 'multi_freq'
    secondary_shape = 'multi_freq'
calibration = QubicCalibration(d)

In [ ]:
hornarray = calibration.get("hornarray")
detarray = calibration.get("detarray")
optics = calibration.get("optics")
primbeam = calibration.get("primbeam")

In [ ]:
print(optics)

In [ ]:
npeaks = (2 * d["synthbeam_kmax"] + 1)**2
ndet = 256 if d["config"] == "TD" else 1024
print(d["MultiBand"], d["npointings"], d["nf_sub"], npeaks)

print("Estimated memory to be used: {:.2f}Gb".format(d["npointings"] * 256 * \
                                               d["nf_sub"] * npeaks/3 * 16 / 1024**3))

In [ ]:
# Load scene
scene = qubic.QubicScene(d)
pointing = qubic.get_pointing(d)

qinstrument = qubic.QubicMultibandInstrument(d)

In [ ]:
noise = {}
noise["samp"] = np.linspace(0,d["npointings"]-1, d["npointings"]-1)
noise["detector"] = qinstrument[0].get_noise_detector(pointing)
noise["photon"] = qinstrument[0].get_noise_photon(pointing, scene)
noise["total"] = qinstrument[0].get_noise(pointing, scene)

In [ ]:
#Detector
plt.plot(noise["samp"], np.mean(noise["detector"], axis = 0), color = "orange",
        label = "detector")
plt.fill_between(noise["samp"], y1 = np.mean(noise["detector"], axis = 0) - \
                 np.std(noise["detector"], axis = 0),
                y2 = np.mean(noise["detector"], axis = 0) + \
                 np.std(noise["detector"], axis = 0), alpha = 0.2, color = "orange")
#Photon
plt.plot(noise["samp"], np.mean(noise["photon"], axis = 0), color = "green",
        label = "photon")
plt.fill_between(noise["samp"], y1 = np.mean(noise["photon"], axis = 0) - \
                 np.std(noise["photon"], axis = 0),
                y2 = np.mean(noise["photon"], axis = 0) + \
                 np.std(noise["photon"], axis = 0), alpha = 0.2, color = "green", )
#Total = detector + noise
plt.plot(noise["samp"], np.mean(noise["total"], axis = 0), color = "darkblue",
        label = "total")
plt.fill_between(noise["samp"], y1 = np.mean(noise["total"], axis = 0) - \
                 np.std(noise["total"], axis = 0),
                y2 = np.mean(noise["total"], axis = 0) + \
                 np.std(noise["total"], axis = 0), alpha = 0.2, color = "darkblue", )
#set 
plt.title("Noise in the {}".format(d["config"]))
plt.xlabel("Sampling")
plt.ylabel("NEP")
plt.legend()

### Fit noise

In [ ]:
muTot, stdTot = norm.fit(np.mean(noise["total"], axis = 1)) 
muDet, stdDet = norm.fit(np.mean(noise["detector"], axis = 1)) 
muPhot, stdPhot = norm.fit(np.mean(noise["photon"], axis = 1)) 

In [ ]:
print(np.shape(noise["total"]),np.shape(np.mean(noise["total"], axis = 1)))
np.mean(noise["total"], axis = 1)

In [ ]:
plt.plot(np.mean(noise["total"], axis = 1), 'ko')
meanvals = np.mean(noise["total"], axis = 1)
maskvals = abs(meanvals) < 3*stdTot
muTot = np.mean(meanvals[maskvals])
print(np.sum(maskvals))

In [ ]:
#histTot = plt.hist(np.mean(noise["total"][maskvals], axis = 1), bins = 30, color = "darkblue",
#        alpha = 0.7, label = "total $\mu, \sigma$ = {:.2e} W/sqrt(Hz),{:.2e}".format(muTot, stdTot))
histTot = plt.hist(np.mean(noise["total"], axis = 1), bins = 30, color = "darkblue",
        alpha = 0.7, label = "total $\mu, \sigma$ = {:.2e} W/sqrt(Hz),{:.2e}".format(muTot, stdTot))
plt.axvline(muTot, color = "darkblue", ls = "--")
histDet = plt.hist(np.mean(noise["detector"], axis = 1), bins = 30, color = "orange",
        alpha = 0.4, label = "detector")
histPhot = plt.hist(np.mean(noise["photon"], axis = 1), bins = 30, color = "green",
        alpha = 0.4, label = "photon")
cc = plt.Circle(( -1e-17 , 20 ), 0.4e-17 ) 
 
#axes.set_aspect( 1 ) 
#plt.add_artist(cc)

# Set
plt.title("Histogram of NEP values in each detector ({})".format(d["config"]))
#plt.ylabel("Frequency (avg. over dets)")
plt.xlabel("NEP")
plt.legend()

### New extraction of noise
The current model of QUBIC in `qubicsoft` for the total NEP consider the following 20 components of the noise:
CMB, atmosphere, windows, block[1-6], 12cmed, plgr, HWP, back-to-back, combin, clspe, dichro (if FI), ndf, 7cmlpe, 6.2cmlpe, 5.6cmlpe

Here I show how to extract the noise for each component using `qubicsoft`


In [ ]:
#Set the parameters for the noise computation. This works with QubicInstrument
noisepar = qinstrument[0].load_NEP_parameters(scene)

In [ ]:
noisepar.names

Compute for one sub-band

In [ ]:
noise_before = qinstrument[0].NEP_before_horns(noisepar, noisepar.nu, 
                                               return_only = True, sampling = pointing)
noise_b2b = qinstrument[0].NEP_horns(noisepar, return_only = True, sampling = pointing)
noise_env = qinstrument[0].NEP_environment(noisepar, noisepar.names, 
                                           return_only = True, sampling = pointing)
noise_comb = qinstrument[0].NEP_combiner(noisepar, 
                                         return_only = True, sampling = pointing)
noise_cs = qinstrument[0].NEP_coldstop(noisepar, 
                                       return_only = True, sampling = pointing)
if d['config'] == 'FI':
    noise_dich = qinstrument[0].NEP_dichroic(noisepar, return_only = True, sampling = pointing)

if d['filter_nu'] == 150e9:
    noise_ndf = qinstrument[0].NEP_neutraldensityfilter(noisepar, 
                                           return_only = True, sampling = pointing)
    noise_lpe1 = qinstrument[0].NEP_lowpassedge(noisepar, noisepar.lpe1,
                                           return_only = True, sampling = pointing)
    noise_lpe2 = qinstrument[0].NEP_lowpassedge(noisepar, noisepar.lpe2,
                                           return_only = True, sampling = pointing)
    
else:
    noise_ndf = qinstrument[0].NEP_lpefilter_220(noisepar, noisepar.indf, 
                                           return_only = True, sampling = pointing)
    noise_lpe1 = qinstrument[0].NEP_lpefilter_220(noisepar, noisepar.lpe1,
                                           return_only = True, sampling = pointing)
    noise_lpe2 = qinstrument[0].NEP_lpefilter_220(noisepar, noisepar.lpe2,
                                           return_only = True, sampling = pointing)
    
noise_last = qinstrument[0].NEP_lastfilter(noisepar,
                                            return_only = True, sampling = pointing)

allnoisearray = [noise_before,noise_b2b,noise_comb,noise_cs,#noise_env,
                 noise_dich,noise_ndf,noise_lpe1,noise_lpe2, noise_last]


#### Confirm a possible bug in NEP_environment

In [ ]:
#fig3 = plt.figure(figsize = (16,6))
#fig3.suptitle('Comparisson btwn Current and New values of the environment NEP_phot ')
#gs = fig3.add_gridspec(1, 5)
#f3_ax1 = fig3.add_subplot(gs[0, :3])
#f3_ax1.set_title('')
#f3_ax1.plot(np.arange(992), noise_env['debug_current'], 'g.-',label = "current version")
#f3_ax1.scatter(np.arange(992), noise_env['debug_new'], c = noise_env['debug_new'], label = "proposal")
#f3_ax1.set_xlabel("numb of detector")
#f3_ax1.set_ylabel(r"environmnt NEP$_{\rm phot}$")
# 
#f3_ax2 = fig3.add_subplot(gs[0, 3:])
#cp = f3_ax2.scatter(qinstrument[0].detector.center.T[0],
#           qinstrument[0].detector.center.T[1],
#          c = noise_env['debug_new'], marker = 's')
#cbar = fig3.colorbar(cp,)
#cbar.set_label('proposal')
#f3_ax1.legend()

#f3_ax2.set_title('')
#f3_ax2.set_xticklabels([])
#f3_ax2.set_yticklabels([])
##f3_ax2.
##qinstrument[0].detector.plot()
##plt.savefig("nep_phot2_env_nobunch.pdf", format = "pdf")

In [ ]:
#print(noisepar.names), len(noise_before["power"]), len(allnoisearray[1:]), len(noisepar.temperatures)

In [ ]:
mp.rcParams["font.size"] = 18

fig2, ax2 = plt.subplots(nrows = 3, ncols = 4, figsize = (16,12))
fig2.suptitle('Mean optical spectral power of the effectively detected radiation (comp before B2B)')
ax2 = ax2.ravel()
ncomp_before_b2b = len(noise_before["power"])
ncomp_after_b2b = len(allnoisearray[1:])
for j in range(ncomp_before_b2b):
    ax2[j].cla()
    plt.axes(ax2[j])
    ax2[j].set_title("{}".format(noisepar.names[j]))
    cp = ax2[j].scatter(qinstrument[0].detector.center.T[0],
               qinstrument[0].detector.center.T[1],
              c = allnoisearray[0]['power'][j], marker = 's')
    cbar = fig2.colorbar(cp,)
    #cbar.set_label('proposal')
    ax2[j].set_xticklabels("")
    ax2[j].set_yticklabels("")

fig, ax = plt.subplots(nrows = 2, ncols = 4, figsize = (16,10))
fig.suptitle('Mean optical spectral power of the effectively detected radiation')
ax = ax.ravel()
for i in range(0,ncomp_after_b2b):
    ax[i].cla()
    plt.axes(ax[i])
    ax[i].set_title("{}".format(noisepar.names[i + ncomp_before_b2b]))
    cp = ax[i].scatter(qinstrument[0].detector.center.T[0],
               qinstrument[0].detector.center.T[1],
              c = allnoisearray[i+1]['power'], marker = 's')
    cbar = fig.colorbar(cp,)
    #cbar.set_label('proposal')
    ax[i].set_xticklabels("")
    ax[i].set_yticklabels("")


### Look at the same filter and different frequencies 

NEP has to evolve with frequency because it changes the power recived?

In [ ]:
noise_per_freq = []#np.zeros((len(qinstrument)))

fig, ax = plt.subplots(nrows = 3, ncols = 4, figsize = (16, 10))
ax = ax.ravel()

fig.suptitle("LPE1 all freqs")
for i in range(len(qinstrument)):
    if d['filter_nu'] == 220e9:
        noise_per_freq.append(qinstrument[i].NEP_lpefilter_220(noisepar, noisepar.lpe1, 
                                           return_only = True, sampling = pointing))
    else:    
        noise_per_freq.append(qinstrument[i].NEP_lowpassedge(noisepar, noisepar.lpe1,
                              return_only = True, sampling = pointing))
    ax[i].cla()
    plt.axes(ax[i])
    cp = ax[i].scatter(qinstrument[i].detector.center.T[0],
               qinstrument[1].detector.center.T[1],
              c = noise_per_freq[i]['power'], marker = 's')
    cbar = fig.colorbar(cp,)
    #cbar.set_label('proposal')
    ax[i].set_xticklabels("")
    ax[i].set_yticklabels("")


In [ ]:
det = 23
plt.plot(noise["total"][det], color = "k", label = "total",
        alpha = 0.3)

for icomp, data in enumerate(noise_before["NEP_array"]):
    plt.plot(data[det], color = "blue", 
             label = "Before horns ({} components)".format(\
                    len(noise_before["NEP_array"])) if icomp == 0 else None)
plt.plot(noise_b2b["NEP_array"][det], color = "red", label = "horns")
plt.plot(noise_env["NEP_array"][det], color = "orange", label = "envelope")
plt.plot(noise_comb["NEP_array"][det], color = "magenta", label = "combiner")
plt.plot(noise_cs["NEP_array"][det], color = "cyan", label = "cold stop")
plt.plot(noise_ndf["NEP_array"][det], color = "brown", label = "NDF")
plt.plot(noise_lpe1["NEP_array"][det], color = "darkblue", label = "LPE1")
plt.plot(noise_dich["NEP_array"][det], color = "brown", label = "Dichroic")
plt.plot(noise_lpe2["NEP_array"][det], color = "darkred", label = "LPE2")
plt.plot(noise_last["NEP_array"][det], color = "darkgreen", label = "LPE5.6")

# Set
plt.xlabel("sample")
plt.ylabel("NEP W /sqrt(Hz)")
plt.title("NEP for detector {} (qsoft index)".format(det))
plt.legend(ncol = 3)

Compare between sub-bands

In [ ]:
freq = 4
noise_before2 = qinstrument[freq].NEP_before_horns(noisepar, noisepar.nu, 
                                               return_only = True, sampling = pointing)
noise_b2b2 = qinstrument[freq].NEP_horns(noisepar, return_only = True, sampling = pointing)
noise_env2 = qinstrument[freq].NEP_environment(noisepar, noisepar.names, 
                                           return_only = True, sampling = pointing)
noise_comb2 = qinstrument[freq].NEP_combiner(noisepar, 
                                         return_only = True, sampling = pointing)
noise_cs2 = qinstrument[freq].NEP_coldstop(noisepar, 
                                       return_only = True, sampling = pointing)
noise_ndf2 = qinstrument[freq].NEP_neutraldensityfilter(noisepar, 
                                                    return_only = True, sampling = pointing)
noise_lpe12 = qinstrument[freq].NEP_lowpassedge(noisepar, noisepar.lpe1,
                                             return_only = True, sampling = pointing)
noise_lpe22 = qinstrument[freq].NEP_lowpassedge(noisepar, noisepar.lpe2,
                                             return_only = True, sampling = pointing)
noise_last2 = qinstrument[freq].NEP_lastfilter(noisepar,
                                            return_only = True, sampling = pointing)

Set the frequencies for QubicMultibandInstrument

In [ ]:
_,_,nus,_,_,_ = qubic.compute_freq(d["filter_nu"], Nfreq = d["nf_sub"], 
             relative_bandwidth = 0.25)

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (18,18),
                                            sharey = True, sharex = True)
fig.subplots_adjust(wspace=0, hspace=0)

ax = ax.ravel()
fig.suptitle('Differences between sub-bands {:.1f}-{:.1f}GHz for each component of the noise (det = {})'.format(
                nus[0]/1e9,nus[1]/1e9, det), 
                 fontsize = 20)

#Components before b2b
ax[0].set_title("Comps. before B2B")
for icomp, data in enumerate(noise_before["NEP_array"]):
    ax[0].plot((data[det] - noise_before2["NEP_array"][icomp][det])/data[det], 
               color = "blue", 
             label = "Before horns ({} components)".format(\
                    len(noise_before["NEP_array"])) if icomp == 0 else None)
ax[0].set_ylim(-1e3, 1e3)
ax[0].set_ylabel("$\Delta$NEP / NEP$_0$")

# back-to-back horns
ax[1].set_title("Back-to-back horns")
ax[1].plot((noise_b2b["NEP_array"]-noise_b2b2["NEP_array"])[det]/noise_b2b["NEP_array"][det], 
           color = "red")
ax[1].set_ylim(-1e3,1e3)

# Envelope
ax[2].set_title("Envelope")
ax[2].plot((noise_env["NEP_array"]-noise_env2["NEP_array"])[det]/noise_env["NEP_array"][det], 
           color = "green")
ax[2].set_ylim(-1e3,1e3)

# Combiner
ax[3].set_title("Combiner")
ax[3].plot((noise_comb["NEP_array"]-noise_comb2["NEP_array"])[det]/noise_comb["NEP_array"][det], 
           color = "darkblue")
ax[3].set_ylim(-1e3,1e3)
ax[3].set_ylabel("$\Delta$NEP / NEP$_0$")

# Cold stop
ax[4].set_title("Cold stop")
ax[4].plot((noise_cs["NEP_array"]-noise_cs2["NEP_array"])[det]/noise_cs["NEP_array"][det], 
           color = "brown")
ax[4].set_ylim(-1e3,1e3)

# Neutral density filter
ax[5].set_title("Neutral density filter")
ax[5].plot((noise_ndf["NEP_array"]-noise_ndf2["NEP_array"])[det], 
           color = "magenta")
#ax[5].set_ylim(-1e3,1e3)

# Low pass edge1
ax[6].set_title("Low pass edge 1")
ax[6].plot((noise_lpe1["NEP_array"]-noise_lpe12["NEP_array"])[det]/noise_lpe1["NEP_array"][det], 
           color = "darkred")
ax[6].set_ylim(-1e3,1e3)
ax[6].set_ylabel("$\Delta$NEP / NEP$_0$")
ax[6].set_xlabel("sample")

# Low pass edge2
ax[7].set_title("Low pass edge 2")
ax[7].plot((noise_lpe2["NEP_array"]-noise_lpe22["NEP_array"])[det]/noise_lpe12["NEP_array"][det], 
           color = "gray")
ax[7].set_ylim(-1e3,1e3)
ax[7].set_xlabel("sample")

# Low pass edge - last filter
ax[8].set_title("Low pass edge - last")
ax[8].plot((noise_last["NEP_array"]-noise_last2["NEP_array"])[det]/noise_last["NEP_array"][det], 
           color = "cyan")
ax[8].set_ylim(-1e3,1e3)
ax[8].set_xlabel("sample")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (18,18),
                      sharey = True, sharex = True)
fig.subplots_adjust(wspace=0, hspace=0)
ax = ax.ravel()
fig.suptitle('Differences between sub-bands {:.1f}-{:.1f}GHz for each component of the noise (avg. over dets)'.format(
                nus[0]/1e9,nus[1]/1e9, det), 
                 fontsize = 20)

#Components before b2b
ax[0].set_title("Comps. before B2B")
for icomp, data in enumerate(noise_before["NEP_array"]):
    ax[0].plot((data - noise_before2["NEP_array"][icomp]).mean(axis = 0)/data.mean(axis = 0), 
               color = "blue", 
             label = "Before horns ({} components)".format(\
                    len(noise_before["NEP_array"])) if icomp == 0 else None)
ax[0].set_ylim(-1e3, 1e3)
ax[0].set_ylabel("$\Delta$NEP / NEP$_0$")
# back-to-back horns
ax[1].set_title("Back-to-back horns")
ax[1].plot((noise_b2b["NEP_array"]-noise_b2b2["NEP_array"]).mean(axis = 0)/ \
           noise_b2b["NEP_array"].mean(axis = 0), 
           color = "red")
ax[1].set_ylim(-1e3,1e3)

# Envelope
ax[2].set_title("Envelope")
ax[2].plot((noise_env["NEP_array"]-noise_env2["NEP_array"]).mean(axis = 0)/\
           noise_env["NEP_array"].mean(axis = 0), 
           color = "green")
ax[2].set_ylim(-1e3,1e3)

# Combiner
ax[3].set_title("Combiner")
ax[3].plot((noise_comb["NEP_array"]-noise_comb2["NEP_array"]).mean(axis = 0)/ \
           noise_comb["NEP_array"].mean(axis = 0), 
           color = "darkblue")
ax[3].set_ylim(-1e3,1e3)
ax[3].set_ylabel("$\Delta$NEP / NEP$_0$")

# Cold stop
ax[4].set_title("Cold stop")
ax[4].plot((noise_cs["NEP_array"]-noise_cs2["NEP_array"]).mean(axis = 0)/ \
           noise_cs["NEP_array"].mean(axis = 0), 
           color = "brown")
ax[4].set_ylim(-1e3,1e3)

# Neutral density filter
ax[5].set_title("Neutral density filter")
ax[5].plot((noise_ndf["NEP_array"]-noise_ndf2["NEP_array"]).mean(axis = 0), 
           color = "magenta")
#ax[5].set_ylim(-1e3,1e3)

# Low pass edge1
ax[6].set_title("Low pass edge 1")
ax[6].plot((noise_lpe1["NEP_array"]-noise_lpe12["NEP_array"]).mean(axis = 0)/ \
           noise_lpe1["NEP_array"].mean(axis = 0), 
           color = "darkred")
ax[6].set_ylim(-1e3,1e3)
ax[6].set_ylabel("$\Delta$NEP / NEP$_0$")
ax[6].set_xlabel("sample")

# Low pass edge2
ax[7].set_title("Low pass edge 2")
ax[7].plot((noise_lpe2["NEP_array"]-noise_lpe22["NEP_array"]).mean(axis = 0)/ \
           noise_lpe12["NEP_array"].mean(axis = 0), 
           color = "gray")
ax[7].set_ylim(-1e3,1e3)
ax[7].set_xlabel("sample")

# Low pass edge - last filter
ax[8].set_title("Low pass edge - last")
ax[8].plot((noise_last["NEP_array"]-noise_last2["NEP_array"]).mean(axis = 0)/ \
           noise_last["NEP_array"].mean(axis = 0), 
           color = "cyan")
ax[8].set_ylim(-1e3,1e3)
ax[8].set_xlabel("sample")

plt.tight_layout()